<a href="https://colab.research.google.com/github/ppiiesle3y1/TLDR/blob/main/AT2_NLP_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#provide huggingface login
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#load libs
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk
import numpy as np

In [ ]:
ds = load_dataset("multi_news")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model_checkpoint="facebook/bart-large-cnn"
metric=load_metric("rouge")

<ipython-input-7-90a1ed71f25c>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=load_metric("rouge")


In [ ]:
ds['train'][0]

{'document': 'National Archives \n \n Yes, it’s that time again, folks. It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. \n \n A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. \n \n Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you’re here, why don’t you sign up to follow us on Twitter. \n \n Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy 

## Preprocessing the raw data






In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_news_length=1024
max_sum_length=128

#tokenize news and summary
def preprocess(dataset):
    document=[doc for doc in dataset["document"]]
    model_inputs=tokenizer(document,max_length=max_news_length,truncation=True)

    with tokenizer.as_target_tokenizer():
        labels=tokenizer(dataset["summary"],max_length=max_sum_length,truncation=True)
    model_inputs["labels"]=labels["input_ids"]
    return model_inputs
    

In [ ]:
preprocess(ds['train'][:2])

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[0, 18285, 26844, 1437, 50118, 1437, 50118, 3216, 6, 24, 17, 27, 29, 14, 86, 456, 6, 5450, 4, 85, 17, 27, 29, 5, 78, 273, 9, 5, 353, 6, 77, 13, 65, 655, 12, 2527, 12, 428, 24062, 1151, 5, 3168, 9, 2298, 852, 6, 663, 8, 4326, 852, 32, 70, 14485, 15, 65, 631, 35, 13499, 4, 1437, 50118, 1437, 50118, 83, 2310, 2935, 15, 5, 121, 4, 104, 4, 4042, 1068, 13, 644, 2323, 5, 22893, 23, 290, 35, 541, 10, 4, 119, 4, 188, 469, 86, 1839, 65, 9, 5, 144, 505, 40617, 15, 141, 5, 866, 24779, 148, 5, 986, 353, 4, 12809, 1635, 32, 13, 23041, 6, 151, 92, 1315, 7, 28, 1412, 6, 309, 7, 9019, 13829, 30, 4614, 1454, 188, 4184, 7948, 6, 1118, 7, 30398, 6, 151, 1315, 355, 11, 902, 4, 20, 5755, 731, 16, 421, 7, 946, 5204, 23, 290, 4, 246, 2153, 1437, 50118, 1437, 50118, 1398, 23, 1018, 5522, 21956, 6, 52, 17, 27, 890, 28, 1839, 3195, 9765, 137, 8, 71, 5, 414, 20238, 5, 22893, 4, 22079, 481, 7, 9832, 12, 179, 2512, 6, 1241, 5, 1450, 2810, 4, 178, 150, 47, 17, 27, 241, 259, 6, 596, 218, 17, 27, 90, 47

In [ ]:
ds_token = ds.map(preprocess, batched=True)

In [ ]:
ds_token

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5622
    })
})

## Training and fine tuning the facebook BART Model


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    f"TLDR/fined-tuned-bart",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size
)

In [ ]:
collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def metrics_eval(pred_eval):
    predictions, labels = pred_eval
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=ds_token["train"],
    eval_dataset=ds_token["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=metrics_eval
)

/content/TLDR/fined-tuned-bart is already a clone of https://huggingface.co/ppiiesle3y/fined-tuned-bart. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import nltk
nltk.download('punkt')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.046600,2.014668,43.548500,16.595400,24.765800,37.600400,129.047800
2,1.837500,2.014337,43.724000,16.568400,24.771200,37.713200,128.982000
3,1.673300,2.006630,43.706500,16.553300,24.758800,37.758600,129.137900
4,1.554900,2.039723,44.047000,16.741700,24.899100,38.048400,129.115100
5,1.484700,2.068926,43.980700,16.664100,24.872000,37.857400,129.022800


TrainOutput(global_step=28110, training_loss=1.7308158252977976, metrics={'train_runtime': 23069.7942, 'train_samples_per_second': 9.747, 'train_steps_per_second': 1.218, 'total_flos': 4.872951408309043e+17, 'train_loss': 1.7308158252977976, 'epoch': 5.0})